In [1]:
import torch
import torch.nn as nn
import numpy as np
import random

from tqdm import tqdm
from common.utils import PickleReadObjectFromLocalPatient, PickleWriteObjectToLocalPatient

Notebook này nhằm mục đích tìm ra các cặp bài viết tương tự của tất cả các bài viết trong tập dữ liệu.

In [2]:
dataset = 'soha'

In [3]:
def load_pretrained_word_embedding():
        object_reader = PickleReadObjectFromLocalPatient()
        embedding_matrix: np.ndarray = object_reader.read(file_name=f"./data/version_1/{dataset}/pretrained/title_abstract_word_embedding.pkl", num_tries=1, wait_time=0.0)
        embedding_matrix: Tensor = torch.tensor(embedding_matrix, dtype=torch.float)
        word_embedding = nn.Embedding.from_pretrained(embeddings=embedding_matrix, padding_idx=0, freeze=False)
        return word_embedding

In [4]:
object_reader = PickleReadObjectFromLocalPatient()
word_to_int = object_reader.read(file_name=f'./data/version_1/{dataset}/object_to_int/word_to_int.pkl')

In [5]:
embedding = load_pretrained_word_embedding()

In [6]:
sample = torch.tensor([45, 46])

In [7]:
x, y = embedding(sample)

In [8]:
cos = nn.CosineSimilarity(dim=0, eps=1e-6)

In [9]:
cos(x, y)

tensor(0.1944, grad_fn=<SumBackward1>)

In [21]:
postid_to_info = object_reader.read(file_name=f'./data/version_1/{dataset}/train/post_id_to_info.pkl')

In [23]:
def find_max_similarity(vector_indices, vectors):
    # sim_matrix = np.zeros((max(vector_indices) + 1, max(vector_indices) + 1))
    sim_dict = {}
    print('done sim matrix')
    for i in tqdm(vector_indices):
        max_sim = 0
        for j in random.sample(vector_indices, 100):
            if i != j:
                sim = cos(torch.tensor(vectors[i]), torch.tensor(vectors[j]))
                if sim > max_sim:
                    max_sim = sim
        sim_dict[i] = j
    return sim_dict

In [24]:
vector_indices = list(postid_to_info.keys())

In [25]:
vectors = np.zeros((max(vector_indices) + 1, 768))

In [26]:
for idx in tqdm(vector_indices):
    vectors[idx] = postid_to_info[idx]['title_bert_encode']

100%|████████████████████████████████████████████████████████████████████████████| 49139/49139 [00:00<00:00, 182124.16it/s]


In [27]:
sim_dict = find_max_similarity(vector_indices, vectors)

done sim matrix


100%|███████████████████████████████████████████████████████████████████████████████| 49139/49139 [05:08<00:00, 159.19it/s]


In [31]:
object_writer = PickleWriteObjectToLocalPatient()

In [32]:
object_writer.write(sim_dict, f'./data/version_1/{dataset}/train/item_to_similar_item.pkl')

True

In [33]:
type(sim_dict)

dict